In [ ]:
import requests
import pandas as pd
import datetime
import os

In [ ]:
def get_binance_klines(symbol="BTCUSDT", interval="1h", limit=1000):
    url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    # Convert to DataFrame
    df = pd.DataFrame(data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
    ])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    return df

def save_to_csv(df, symbol, interval):
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    filename = f"{symbol}_{interval}_{today}.csv"
    folder = "binance_data"
    os.makedirs(folder, exist_ok=True)
    path = os.path.join(folder, filename)
    df.to_csv(path, index=False)
    print(f"✅ Saved: {path}")

# Main block
if __name__ == "__main__":
    symbols = ["BTCUSDT", "ETHUSDT", "SOLUSDT", "XRPUSDT"]
    interval = "1h"  # You can change this to "15m", "1d", etc.
    limit = 1000     # Max candles per request

    for symbol in symbols:
        try:
            df = get_binance_klines(symbol, interval, limit)
            save_to_csv(df, symbol, interval)
        except Exception as e:
            print(f"❌ Error fetching {symbol}: {e}")


In [ ]:
def get_binance_klines(symbol="BTCUSDT", interval="1h", limit=1000):
    url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    # Convert to DataFrame
    df = pd.DataFrame(data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
    ])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    return df

def save_to_csv(df, symbol, interval):
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    filename = f"{symbol}_{interval}_{today}.csv"
    folder = "binance_data"
    os.makedirs(folder, exist_ok=True)
    path = os.path.join(folder, filename)
    df.to_csv(path, index=False)
    print(f"✅ Saved: {path}")

# Main block
if __name__ == "__main__":
    symbols = ["BTCUSDT", "ETHUSDT", "SOLUSDT", "XRPUSDT"]
    interval = "1h"  # You can change this to "15m", "1d", etc.
    limit = 1000     # Max candles per request

    for symbol in symbols:
        try:
            df = get_binance_klines(symbol, interval, limit)
            save_to_csv(df, symbol, interval)
        except Exception as e:
            print(f"❌ Error fetching {symbol}: {e}")

In [ ]:
import requests
import pandas as pd
import datetime as dt
import time
import os

def interval_to_milliseconds(interval: str) -> int:
    unit = interval[-1]
    amount = int(interval[:-1])
    conversions = {"m": 60_000, "h": 3_600_000, "d": 86_400_000}
    return amount * conversions[unit]

def date_to_milliseconds(date_str: str) -> int:
    dt_obj = dt.datetime.strptime(date_str, "%Y-%m-%d")
    return int(dt_obj.timestamp() * 1000)

def get_binance_klines(symbol, interval, start_date, end_date=None):
    url = "https://api.binance.com/api/v3/klines"
    start_ts = date_to_milliseconds(start_date)
    end_ts = date_to_milliseconds(end_date) if end_date else int(time.time() * 1000)
    limit = 1000
    df_list = []
    step = interval_to_milliseconds(interval) * limit

    while start_ts < end_ts:
        params = {
            "symbol": symbol,
            "interval": interval,
            "limit": limit,
            "startTime": start_ts,
            "endTime": min(start_ts + step, end_ts)
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        if not data:
            break  # No more data

        df = pd.DataFrame(data, columns=[
            "timestamp", "open", "high", "low", "close", "volume",
            "close_time", "quote_asset_volume", "number_of_trades",
            "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
        ])
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
        df_list.append(df)

        start_ts = data[-1][0] + interval_to_milliseconds(interval)  # avoid duplication

        time.sleep(0.25)  # API rate limit buffer

    return pd.concat(df_list, ignore_index=True)

def save_to_csv(df, symbol, interval):
    today = dt.datetime.now().strftime("%Y-%m-%d")
    filename = f"{symbol}_{interval}_{today}.csv"
    folder = "binance_data"
    os.makedirs(folder, exist_ok=True)
    path = os.path.join(folder, filename)
    df.to_csv(path, index=False)
    print(f"✅ Saved: {path}")

# Main block
if __name__ == "__main__":
    symbols = ["BTCUSDT", "ETHUSDT", "SOLUSDT", "XRPUSDT"]
    interval = "1h"
    start_date = "2020-01-01"
    end_date = None  # Or set like "2024-06-01"

    for symbol in symbols:
        try:
            df = get_binance_klines(symbol, interval, start_date, end_date)
            save_to_csv(df, symbol, interval)
        except Exception as e:
            print(f"❌ Error fetching {symbol}: {e}")
